In [3]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import json
import seaborn as sbrn
import urllib2

%matplotlib inline

In [4]:
whd_whisard = pd.read_csv('/Users/ash/Downloads/wage theft datasets/whd_whisard.csv', low_memory=False)


In [6]:
whd_whisard.shape


(223519, 110)

In [7]:
pd.set_option('max_columns', 50)
whd_whisard.sample(5)

,case_id,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,naics_code_description,case_violtn_cnt,cmp_assd_cnt,ee_violtd_cnt,bw_atp_amt,ee_atp_cnt,findings_start_date,findings_end_date,flsa_violtn_cnt,flsa_repeat_violator,flsa_bw_atp_amt,flsa_ee_atp_cnt,flsa_mw_bw_atp_amt,flsa_ot_bw_atp_amt,flsa_15a3_bw_atp_amt,flsa_cmp_assd_amt,sca_violtn_cnt,...,pca_bw_atp_amt,pca_ee_atp_cnt,flsa_smwap_violtn_cnt,flsa_smwap_bw_atp_amt,flsa_smwap_ee_atp_cnt,flsa_smwft_violtn_cnt,flsa_smwft_bw_atp_amt,flsa_smwft_ee_atp_cnt,flsa_smwl_violtn_cnt,flsa_smwl_bw_atp_amt,flsa_smwl_ee_atp_cnt,flsa_smwmg_violtn_cnt,flsa_smwmg_bw_atp_amt,flsa_smwmg_ee_atp_cnt,flsa_smwsl_violtn_cnt,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,ld_dt
108853,1604446,Farmington Marriot,NaN,15 Farm Springs Road,Farmington,CT,6032.0,721110,Hotels (except Casino Hotels) and Motels,17,0.0,16,4834.80,16,2009-03-01,2011-03-01,17,NaN,4834.80,16,0.00,4834.80,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241
34070,1357474,Hillside Care Center,"Buckeye Family Mental Health, Inc.",339 North Prospect,Marion,OH,43302.0,62311,Nursing Care Facilities,16,0.0,15,3801.87,15,2002-04-03,2004-03-31,16,NaN,3801.87,15,0.00,3801.87,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241
117807,1590781,"Quatrro Mortgage Solutions, Inc","Quatrro Mortgage Solutions, Inc","1350 Old Bayshore Highway, Suite 630",Burlingame,CA,94010.0,523910,Miscellaneous Intermediation,17,0.0,16,16533.79,16,2010-01-28,2010-08-31,17,NaN,16533.79,16,0.00,16533.79,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241
61614,1758947,United States Postal Service (USPS),United States Postal Service (USPS),74801 Hovley Lane East,Palm Desert,CA,92260.0,491110,Postal Service,1,0.0,1,431.45,1,2015-01-05,2015-01-27,1,NaN,431.45,1,431.45,0.00,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2016-02-19 01:00:04.033708
43379,1507826,La Mixteca,Abarrotes Mixteca Corp,5000 Grand Avenue- Section E,Maspeth,NY,11378.0,311830,Tortilla Manufacturing,0,0.0,0,0.00,0,2008-01-07,2008-03-28,0,NaN,0.00,0,0.00,0.00,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241


In [8]:
# need lats and longs based on street addy and zip code and state code. Any missing values?
whd_whisard['zip_cd'].isnull().sum()


19

In [9]:
#set zip to int
#whd_whisard[whd_whisard['zip_cd'].isnull()]['zip_cd'] = np.nan
whd_whisard['zip_cd'].astype('int64', raise_on_error=False)

0         48917.0
1         27534.0
2          2067.0
3         78542.0
4         32308.0
5         72042.0
6          8360.0
7         44203.0
8         78596.0
9         19902.0
10        78521.0
11        85023.0
12        33801.0
13        34239.0
14        75202.0
15        34652.0
16        85304.0
17        28365.0
18        16115.0
19        29926.0
20        77563.0
21         6103.0
22        92021.0
23        63109.0
24        72756.0
25        60056.0
26         5404.0
27        34453.0
28        95236.0
29        34950.0
           ...   
223489    78621.0
223490    95820.0
223491     6475.0
223492    90240.0
223493    18342.0
223494    95932.0
223495    44481.0
223496    33880.0
223497    78580.0
223498    63601.0
223499    68127.0
223500    68508.0
223501    34266.0
223502    94583.0
223503    50049.0
223504    97402.0
223505    54479.0
223506    40202.0
223507    26250.0
223508    41018.0
223509    98264.0
223510    41005.0
223511    90023.0
223512     6103.0
223513    

In [12]:
# Was planning on using Google Maps API to get detialed lats and longs based on street addy, but the rate limit is
# 2500 requests a day, 10 requests/sec; I won't be able to d'load all this within 10 days or so for 223,519 records.
# Plan B: Got a dataset from US Census Bureau of zip codes to lat longs, that'll do for now. 
# remember, (Got 19 zips missing)

#read in the filw
census_loc = pd.read_table('/Users/ash/Downloads/wage theft datasets/other datasets/2015_Gaz_zcta_national.txt')

In [13]:
census_loc.drop(['ALAND','AWATER','ALAND_SQMI','AWATER_SQMI'], axis=1, inplace=True)
census_loc.sample(5)

,GEOID,INTPTLAT,INTPTLONG
29767,88130,34.129177,-103.299416
2656,10456,40.829886,-73.908121
29199,85325,33.992347,-113.911284
21357,62215,38.508296,-89.608858
17525,52405,41.983384,-91.746369


In [14]:
whd_whisard=whd_whisard.merge(census_loc, how='left', left_on='zip_cd', right_on='GEOID')

In [15]:
whd_whisard.rename(columns={'INTPTLAT':'LATITUDE'}, inplace=True)
whd_whisard.rename(columns={'INTPTLONG':'LONGITUDE'}, inplace=True)

In [16]:
whd_whisard.columns.values[112] = 'LONGITUDE'

In [17]:
whd_whisard.columns #hallelujah!

Index([u'case_id', u'trade_nm', u'legal_name', u'street_addr_1_txt', u'cty_nm',
       u'st_cd', u'zip_cd', u'naic_cd', u'naics_code_description',
       u'case_violtn_cnt',
       ...
       u'h2b_violtn_cnt', u'h2b_bw_atp_amt', u'h2b_ee_atp_cnt',
       u'sraw_violtn_cnt', u'sraw_bw_atp_amt', u'sraw_ee_atp_cnt', u'ld_dt',
       u'GEOID', u'LATITUDE', u'LONGITUDE'],
      dtype='object', length=113)

In [ ]:
whd_whisard.to_csv('/Users/ash/Downloads/wage theft datasets/whd_whisard_latlong.csv')

In [13]:
#Occupational Employment Statistics, 2015, from http://www.bls.gov/oes/
oes = pd.read_excel('/Users/ash/Downloads/wage theft datasets/other datasets/oesm15all/all_data_M_2015.xlsx')



In [101]:
# we found out below that OES data from 2015 barely matches 2% of NAICS codes in WHD_WHISARD. So trying an older OES dataset (2007)
# http://www.bls.gov/oes/tables.htm (THIS IS 2007)
oes_2007_1 = pd.read_excel('/Users/ash/Downloads/oesm07in4/nat3d_may2007_dl.xls')
oes_2007_2 = pd.read_excel('/Users/ash/Downloads/oesm07in4/nat4d_May2007_dl.xls')
oes_2007_3 = pd.read_excel('/Users/ash/Downloads/oesm07in4/nat5d_may2007_dl.xls')

In [103]:
oes_2007 = pd.concat([oes_2002_1,oes_2002_2,oes_2002_3], axis=0)

In [127]:
(len(set(whd_whisard['naic_cd']) & set(oes_2007['NAICS'])) / float(len(set(whd_whisard['naic_cd']))))
#wow, NOT A SINGLE CODE MATCH! Dunno what's going on with the NAICS codes in OES datasets

0.0

In [31]:
oes.sample(5)
#interesting fields are:
#"naics"
#"own_code" (1= Federal Government; 2= State Government; 3= Local Government; 123= Federal, State, and Local Government; 235=Private, State, and Local Government; 35 = Private and Local Government; 5= Private; 57=Private, Local Government Gambling Establishments (Sector 71), and  Local Government Casino Hotels (Sector 72); 58= Private plus State and Local Government Hospitals; 59= Private and Postal Service; 1235= Federal, State, and Local Government and Private Sector )
#"occ code" (7 digit 'standard' occ code or OES specific code for occupaton)
#"occ title"
# tot_emp (rounded to nearest 10)
# emp_prse (% rlative standard err for employment)Relative Standard Error (RSE) is a measure of the reliability of a statistic; the smaller the relative standard error, the more precise the estimate
# jobs_100 (# of jobs in given occ./1000 jobs in given area)
# loc_quotient (represents the ratio of an occupation’s share of employment in a given area to that occupation’s share of employment in the U.S. as a whole. For example, an occupation that makes up 10 percent of employment in a specific metropolitan area compared with 2 percent of U.S. employment would have a location quotient of 5 for the area in question. Only available for the state and MSA estimates; otherwise this column is blank.)
# pct_total (Percent of industry employment in the given occupation. Percents may not total to 100 due to occupational data not reported. Only available for the national industry estimates; otherwise this column is blank. )
# h_mean (mean hourly wage)
# a_mean (mean annual wage)
# mean_prse (Percent relative standard error for the mean wage. Relative Standard Error (RSE) is a measure of the reliability of a statistic; the smaller the relative standard error, the more precise the estimate.)
# all the percentile stats for these two fields


# *  = indicates that a wage estimate is not available
# **  = indicates that an employment estimate is not available
# #  = indicates a wage equal to or greater than $90.00 per hour or $187,200 per year 

,area,area_title,area_type,naics,naics_title,own_code,occ code,occ title,group,tot_emp,emp_prse,jobs_1000,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
288503,29540,"Lancaster, PA",4,000000,Cross-industry,1235,51-3093,Food Cooking Machine Operators and Tenders,detail,150,8.3,0.634,2.53,NaN,16.19,33680,6.1,11.01,13.99,16.45,18.59,21.66,22890,29100,34210,38670,45050,NaN,NaN
149597,99,U.S.,1,622100,General Medical and Surgical Hospitals,5,43-9110,Statistical Assistants,broad,760,4.1,NaN,NaN,0.02,20.59,42830,1.9,14.25,16.54,19.99,23.7,28.77,29640,34410,41590,49290,59840,NaN,NaN
27535,99,U.S.,1,238200,Building Equipment Contractors,5,51-8031,Water and Wastewater Treatment Plant and Syste...,detail,**,**,NaN,NaN,**,21.92,45600,7,17.49,19.57,21.16,22.75,23.7,36370,40710,44020,47320,49300,NaN,NaN
26468,99,U.S.,1,238100,"Foundation, Structure, and Building Exterior C...",5,49-3022,Automotive Glass Installers and Repairers,detail,500,17,NaN,NaN,0.06,16.88,35120,3.9,10.7,13.1,16.63,20.76,23.6,22250,27240,34590,43180,49080,NaN,NaN
241442,16540,"Chambersburg-Waynesboro, PA",4,000000,Cross-industry,1235,13-1161,Market Research Analysts and Marketing Special...,detail,60,12.7,0.976,0.27,NaN,27.7,57620,5.6,15,20.42,24.5,29.95,46.16,31210,42470,50960,62290,96010,NaN,NaN


In [64]:
#how many naics codes in whd_whisard that are NOT in oes?
len(set(whd_whisard['naic_cd']) & set(oes['naics']))

44

In [68]:
1- (len(set(whd_whisard['naic_cd']) & set(oes['naics'])) / float(len(set(whd_whisard['naic_cd']))))
#98% of NAICS codes in whd_whisard are NOT in oes stats! Big problem.

0.9805309734513274

In [70]:
whd_whisard.sample(5)

,case_id,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,naics_code_description,case_violtn_cnt,cmp_assd_cnt,ee_violtd_cnt,bw_atp_amt,ee_atp_cnt,findings_start_date,findings_end_date,flsa_violtn_cnt,flsa_repeat_violator,flsa_bw_atp_amt,flsa_ee_atp_cnt,flsa_mw_bw_atp_amt,flsa_ot_bw_atp_amt,flsa_15a3_bw_atp_amt,flsa_cmp_assd_amt,sca_violtn_cnt,...,flsa_smwap_bw_atp_amt,flsa_smwap_ee_atp_cnt,flsa_smwft_violtn_cnt,flsa_smwft_bw_atp_amt,flsa_smwft_ee_atp_cnt,flsa_smwl_violtn_cnt,flsa_smwl_bw_atp_amt,flsa_smwl_ee_atp_cnt,flsa_smwmg_violtn_cnt,flsa_smwmg_bw_atp_amt,flsa_smwmg_ee_atp_cnt,flsa_smwsl_violtn_cnt,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt,ld_dt,GEOID,LATITUDE,LONGITUDE
109326,1606010,Outback Steakhouse of Florida (Brandon),"OSI Restaurant Partners, LLC",2020 W Brandon Blvd,Brandon,FL,33511.0,722110,Full-Service Restaurants,0,0.0,0,0.00,0,2009-02-02,2011-02-02,0,NaN,0.00,0,0.00,0.00,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241,33511.0,27.909778,-82.295738
91544,1572287,Almeida Cement Pumping,Almeida Concrete Pumping Corp.,Jobsite: 314 East 100th Street,Jamaica,NY,11435.0,333913,Measuring and Dispensing Pump Manufacturing,9,0.0,4,4542.80,4,2009-11-22,2010-02-06,0,NaN,0.00,0,0.00,0.00,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241,11435.0,40.701229,-73.809640
16913,1342713,Classic Scapes,"Belco, Inc",2828 Simpson Ln.,Carrollton,TX,75006.0,56173,Landscaping Services,80,0.0,80,58166.23,80,2002-03-15,2004-03-15,80,NaN,58166.23,80,0.00,58166.23,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241,75006.0,32.962141,-96.898585
166140,1719026,Captain Jim's Seafood Restaurant,Captain Jim's Seafood Restaurant Inc.,12950 West Dixie Highway,Miami,FL,33161.0,722110,Full-Service Restaurants,35,0.0,27,39985.00,27,2012-02-20,2014-02-16,35,NaN,39985.00,27,23241.12,16743.88,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241,33161.0,25.893664,-80.181539
38015,1485670,Plastic,"Gly, Inc.",1100 S. San Pedro Street,Los Angeles,CA,90015.0,315232,Women's and Girls' Cut and Sew Blouse and Shir...,1,0.0,0,0.00,0,2005-07-03,2007-07-02,1,NaN,0.00,0,0.00,0.00,0.0,0.0,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,2015-02-20 01:00:06.112241,90015.0,34.039378,-118.266300


* in order to merge into the whd_whisard dataset, which contains violations only at the naics code level, we’re going to have to aggregate the oes stats by making the following assumptions
* since the main obj. is to identify “at risk” populations, and most violations are min. wage type violations that the DOL is specifically interested in pursuing, we’re going to take into account only those occ. codes where 90% of the employed labor force is under $15/hour. 
      * aggregation of the oes dataset shall proceed as follows:
            * sum the total employed
            * average the RSE of total employed
            * average all other derived measures that don’t involve a direct count
            * it’ll be nice to capture the occ. codes affected as a result of this aggregation
**End Result**: OES subset with one row per affected naics code that contains summary numbers that can be merged into whd_whisard

In [57]:
#first, keep only occs where 90% pop is <$15/hr 
oes_lowincm = oes[oes['h_pct90']<15]
# wow, this reduces the dataset from 439,942 to 27,593 rows! Doubt this'll cover all the codes in whd_whisard.

(27593, 29)

In [131]:
#exploring Quarterly Census of Employment and Wages API
#http://data.bls.gov/cew/doc/access/data_access_examples.htm#PYTHON

import qcew_python_2x_example as qcew

#this API seems to be broken, nm!

In [134]:
#csv_rests=qcew.qcewGetIndustryData('2013','1','722110') #confirmed bad API (URL)HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
#The last 30x error message was:
#Moved Permanently

HTTPError: HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently

In [82]:
# trying yet another source: http://www2.census.gov/econ/susb/data/2013/us_6digitnaics_2013.xlsx
# 2013 first

link = 'http://www2.census.gov/econ/susb/data/2013/us_6digitnaics_2013.xlsx'
#socket = urllib2.urlopen(link)
#xd = pd.ExcelFile(socket)
#xd.sheet_names

SUSB2013=pd.read_excel(link, skiprows=5)


In [87]:
SUSB2013.sample(5)

,NAICS CODE,NAICS DESCRIPTION,ENTERPRISE EMPLOYMENT SIZE,NUMBER OF FIRMS,NUMBER OF ESTABLISHMENTS,EMPLOYMENT,EMPLOYMENT RANGE FLAG,EMPLOYMENT NOISE FLAG,"ANNUAL PAYROLL ($1,000)",ANNUAL PAYROLL NOISE FLAG
600,212321,Construction Sand and Gravel Mining,05: <20,1097.0,1112.0,5230.0,NaN,G,267662.0,G
6633,33611,Automobile and Light Duty Motor Vehicle Manufa...,08: <500,191.0,191.0,2730.0,NaN,G,131880.0,G
15033,562111,Solid Waste Collection,09: 500+,55.0,2536.0,96403.0,NaN,G,5299279.0,G
3073,3219,Other Wood Product Manufacturing,03: 5-9,1490.0,1492.0,10042.0,NaN,G,296842.0,G
1112,23711,Water and Sewer Line and Related Structures Co...,09: 500+,58.0,225.0,23578.0,NaN,G,1513338.0,G


In [85]:
SUSB2013.columns

Index([u' NAICS   CODE ', u'NAICS DESCRIPTION', u'ENTERPRISE EMPLOYMENT SIZE',
       u'NUMBER OF FIRMS', u'NUMBER OF ESTABLISHMENTS', u'EMPLOYMENT',
       u'EMPLOYMENT RANGE FLAG', u'EMPLOYMENT NOISE FLAG',
       u'ANNUAL PAYROLL ($1,000)', u'ANNUAL PAYROLL NOISE FLAG'],
      dtype='object')

In [88]:
#okay, check NAICS code coverage with whd_whisard
(len(set(whd_whisard['naic_cd']) & set(SUSB2013[' NAICS   CODE '])) / float(len(set(whd_whisard['naic_cd']))))
#78% coverage, much better than OES. Trouble is these #s are by exact year, but whd_whisard spans years... 
# might have to do a first pass irrespective of date and then load different #s based on "findings end date" and SUSBprior years

0.7836283185840708

In [18]:
#try based on older NAICS codes (whd_whisard codes might be older)
link = 'http://www2.census.gov/econ/susb/data/2007/us_6digitnaics_empl_2007.xls'

SUSB2007 = pd.read_excel(link, skiprows=5)
SUSB2007.sample(5)

,NAICS,Unnamed: 1,EMPLOYMENT,NUMBER,NUMBER OF,Unnamed: 5,EMPLOYMENT,EMPLOYMENT.1,PAYROLL,PAYROLL.1,RECEIPTS,RECEIPTS.1
18787,812191,Diet and Weight Reducing Centers,7: 100-499,23,181,4642,NaN,H,64645,G,227725,G
13826,523120,Securities Brokerage,3: 5-9,970,994,6308,NaN,H,727666,H,1478737,G
5292,331319,Other Aluminum Rolling and Drawing,5: <20,46,46,233,NaN,H,10099,H,74931,H
6882,3344,Semiconductor and Other Electronic Component M...,4: 10-19,624,624,8632,NaN,G,379195,G,1368231,G
8025,337129,"Wood Television, Radio, and Sewing Machine Cab...",8: <500,272,273,1684,NaN,G,47060,G,172675,G


In [19]:
(len(set(whd_whisard['naic_cd']) & set(SUSB2007['NAICS'])) / float(len(set(whd_whisard['naic_cd']))))
#yup, even better coverage than 2013 dataset. We use this one!

0.8668141592920354

In [97]:
#and just checking 2002, real quick (http://www2.census.gov/econ/susb/data/2002/us_6digitnaics_empl_2002.xls)
link = 'http://www2.census.gov/econ/susb/data/2002/us_6digitnaics_empl_2002.xls'

SUSB2002 = pd.read_excel(link, skiprows=5)
SUSB2002.sample(5)

,NAICS CODE,NAICS DESCRIPTION,DATA TYPE,TOTAL,Employment Size of the Enterprise,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
2878,327999,All other miscellaneous nonmetallic mineral pr...,Employment,10997,0,206,453,1066,1725,2835,2057,6617,4380
8962,562211,Hazardous waste treatment & disposal,Establishments,572,37,131,53,64,285,82,41,408,164
5131,421940,"Jewelry/watch/silverware/precious stone, & pre...",Firms,8607,1055,5190,1319,592,8156,377,52,8585,22
3789,334111,Electronic computer mfg,"Annual Payroll ($1,000)",4011518,9270,11976,23564,29373,74183,209632,257354,541169,3470349
4836,42139,Other construction materials whsle,Firms,2568,251,1157,474,284,2166,264,79,2509,59


In [99]:
(len(set(whd_whisard['naic_cd']) & set(SUSB2002['NAICS CODE'])) / float(len(set(whd_whisard['naic_cd']))))
#nope, stick with 2007

0.7606194690265486

#### Preparing SUSB 2007 for merge into whd_whisard:
(data definitions are [here](http://www.census.gov/econ/susb/definitions.html)):
+ For each NAICS, aggregate
    ++ # of estalishments (A single physical location where business is conducted or where services or industrial operations are performed)  
    
    ++ # employment  
    
    ++ PAYROLL  
    
    
    ++ RECEIPTS (Estimated Receipts - Receipts (net of taxes collected from customers or clients) are defined as operating revenue for goods produced or distributed, or for services provided. Receipts excludes local, state, and federal sales and other taxes collected from customers or clients and paid directly to a tax agency.)
    

In [20]:
SUSB2007.columns.rename({'Unnamed: 1':'NAICS_desc',
                        'NUMBER':'num_firms',
                        'NUMBER OF':'num_establishments',
                        'Unnamed: 5':'num_employed'},inplace=True)
#SUSB2007.columns

In [21]:
SUSB2007.columns.values[1]='NAICS_desc'
SUSB2007.columns.values[3]='num_firms'
SUSB2007.columns.values[4]='num_establishments'
SUSB2007.columns.values[5]='num_employed'




In [22]:
SUSB2007.columns

Index([             u'NAICS',         u'NAICS_desc',        u'EMPLOYMENT ',
                u'num_firms', u'num_establishments',       u'num_employed',
               u'EMPLOYMENT',       u'EMPLOYMENT.1',            u'PAYROLL',
                u'PAYROLL.1',           u'RECEIPTS',         u'RECEIPTS.1'],
      dtype='object', name={u'Unnamed: 5': u'num_employed', u'NUMBER OF': u'num_establishments', u'Unnamed: 1': u'NAICS_desc', u'NUMBER': u'num_firms'})

In [23]:
#remove NaNs by replacing with 0s (for aggregation)
agg_cols = ['num_firms','num_establishments','num_employed','PAYROLL','RECEIPTS']

In [ ]:
SUSB2007.drop(SUSB2007.iloc[0,:], axis=1)

In [152]:
#first, have to aggregate SUSB2007 into a single row per NAICS. 
#total all counts and average any derived stats
SUSB2007_grps = SUSB2007.groupby(by='NAICS')[agg_cols].aggregate(sum)

In [155]:
SUSB2007

(19264, 12)